Momentum investing is investing in stocks that have increased in price the most. 

Here, we will select the 50 stocks with highest price momentum. Then, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. (From project 1)

In [14]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf

In [25]:
# pandas read html
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# clean data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*2)

# stacking the data to make it easier to work with
# use future_stack=True to avoid the future warning
df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack(future_stack=True)

df

[*********************100%%**********************]  503 of 503 completed


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2022-08-01 A       131.583328  133.429993  135.229996  133.259995  133.509995   
           AAL      14.280000   14.280000   14.320000   13.520000   13.670000   
           AAPL    159.703171  161.509995  163.589996  160.889999  161.009995   
           ABBV    129.740509  140.220001  142.839996  139.149994  141.509995   
           ABNB    111.199997  111.199997  113.959999  107.480003  110.000000   
...                       ...         ...         ...         ...         ...   
2024-07-26 XYL     140.839996  140.839996  142.130005  137.820007  138.479996   
           YUM     128.050003  128.050003  129.039993  127.410004  127.690002   
           ZBH     111.290001  111.290001  112.279999  110.230003  110.790001   
           ZBRA    325.980011  325.980011  330.970001  323.000000  326.500000   
           ZTS     179.839996  179.839996  183.850006  179.259995  182.259995   

Price                  Volume  
Date       Ticker              
2022-08-01 A        1081700.0  
           AAL     32730800.0  
           AAPL    67829400.0  
           ABBV     8523900.0  
           ABNB     6019500.0  
...                       ...  
2024-07-26 XYL      1074100.0  
           YUM      1874400.0  
           ZBH      1399400.0  
           ZBRA      458700.0  
           ZTS      2437300.0  

[251500 rows x 6 columns]

In [57]:
# we want to pull price in 1 yr stock return
tickers_list = df.index.get_level_values(1).unique()
AllYF = yf.Tickers(' '.join(tickers_list)).tickers
# this is a replacement for year1ChangePercent
# yeet = AllYF['AAPL'].info['52WeekChange']

In [5]:
df_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

data_list = [
    [
        ticker,
        values.info.get('previousClose', None), 
        values.info.get('52WeekChange', None),
        'N/A'
    ]
    for ticker, values in AllYF.items()
]

final_df = pd.DataFrame(data=data_list, columns=df_columns)
# 2:02

In [6]:
# inplace will directly modify final_df
final_df.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
# get the top 50 results (50 highest returns)
final_df = final_df[:50]
# reset the index to start from 0
final_df.reset_index(drop=True, inplace=True)

Ngl, this is really lame. We will now create a more realistic momentum strategy with 1, 3, 6, and 12 month returns.

- High-quality m-stocks show "slow and steady" outperformance over long periods of time.
- Low-quality m-stocks might not show any momentum for a long time, then surge upwards.


In [63]:
Ahqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

Ahqm_df = pd.DataFrame(columns=Ahqm_columns)

today = datetime.now()
year1 = (today - relativedelta(years=1)).strftime('%Y-%m-%d')
month6 = (today - relativedelta(months=6)).strftime('%Y-%m-%d')
month3 = (today - relativedelta(months=3)).strftime('%Y-%m-%d')
month1 = (today - relativedelta(months=1)).strftime('%Y-%m-%d')

data_list_full = [
    [
        ticker,
        values.info.get('previousClose', None),
        'N/A',
        values.info.get('previousClose', None) - values.history(start=year1, end=today)['Close'].to_numpy()[0],
        'N/A',
        values.info.get('previousClose', None) - values.history(start=month6, end=today)['Close'].to_numpy()[0],
        'N/A',
        values.info.get('previousClose', None) - values.history(start=month3, end=today)['Close'].to_numpy()[0],
        'N/A',
        values.info.get('previousClose', None) - values.history(start=month1, end=today)['Close'].to_numpy()[0],
        'N/A'
    ]
    for ticker, values in AllYF.items()
]

Ahqm_df = pd.DataFrame(data=data_list_full, columns=Ahqm_columns)
Ahqm_df

,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,137.32,N/A,15.550003,N/A,4.489998,N/A,-2.269996,N/A,9.919998,N/A
1,AAL,10.62,N/A,-6.130000,N/A,-4.320000,N/A,-3.360000,N/A,-0.420000,N/A
2,AAPL,217.96,N/A,21.510003,N/A,26.230004,N/A,44.460000,N/A,1.210000,N/A
3,ABBV,185.16,N/A,35.579998,N/A,21.249996,N/A,23.639996,N/A,14.790005,N/A
4,ABNB,140.10,N/A,-12.090002,N/A,-12.599997,N/A,-22.150000,N/A,-11.530005,N/A
...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,140.84,N/A,28.090000,N/A,27.479999,N/A,8.250004,N/A,9.310001,N/A
499,YUM,128.05,N/A,-9.619998,N/A,-2.500003,N/A,-15.140002,N/A,-1.649997,N/A
500,ZBH,111.29,N/A,-26.859994,N/A,-11.629998,N/A,-10.419999,N/A,5.330001,N/A
501,ZBRA,325.98,N/A,18.020009,N/A,70.050007,N/A,25.969990,N/A,19.770009,N/A


In [ ]:
def calc_ticker_data(ticker, values):
    previousClose = values.info.get('previousClose', None)
    year1_price = values.history(start=year1, end=today)['Close'].to_numpy()[0]
    month6_price = values.history(start=month6, end=today)['Close'].to_numpy()[0]
    month3_price = values.history(start=month3, end=today)['Close'].to_numpy()[0]
    month1_price = values.history(start=month1, end=today)['Close'].to_numpy()[0]
    

In [24]:
today = datetime.now()
year1 = (today - relativedelta(years=1)).strftime('%Y-%m-%d')
year1_price = df.xs((year1, 'AAPL'), level=[0, 1])['Close']
print(year1_price)

KeyError: '2023-07-29'

In [55]:
msft = yf.Ticker("MSFT")

today = datetime.now()
year1 = (today - relativedelta(years=1)).strftime('%Y-%m-%d')

msft.info.get('previousClose', None) - msft.history(start=year1, end=today)['Close'].to_numpy()[0]

msft.info.get('previousClose', None)
msft.history(start=year1, end=today)['Close']

Date
2023-07-31 00:00:00-04:00    333.312622
2023-08-01 00:00:00-04:00    333.729340
2023-08-02 00:00:00-04:00    324.957977
2023-08-03 00:00:00-04:00    324.124481
2023-08-04 00:00:00-04:00    325.235779
                                ...    
2024-07-23 00:00:00-04:00    444.850006
2024-07-24 00:00:00-04:00    428.899994
2024-07-25 00:00:00-04:00    418.399994
2024-07-26 00:00:00-04:00    425.269989
2024-07-29 00:00:00-04:00    426.730011
Name: Close, Length: 251, dtype: float64

In [65]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming AllYF is your data source dictionary containing ticker and values
# AllYF = {...}

# Define the columns for the DataFrame
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

# Initialize the DataFrame
hqm_df = pd.DataFrame(columns=hqm_columns)

# Define date ranges
today = datetime.now()
year1 = (today - relativedelta(years=1)).strftime('%Y-%m-%d')
month6 = (today - relativedelta(months=6)).strftime('%Y-%m-%d')
month3 = (today - relativedelta(months=3)).strftime('%Y-%m-%d')
month1 = (today - relativedelta(months=1)).strftime('%Y-%m-%d')

# Function to process each ticker
def process_ticker(ticker, values):
    try:
        stock_price = values.info.get('previousClose', None)
        one_year_return = (stock_price - values.history(start=year1, end=today)['Close'].to_numpy()[0]) / values.history(start=year1, end=today)['Close'].to_numpy()[0]
        six_month_return = (stock_price - values.history(start=month6, end=today)['Close'].to_numpy()[0]) / values.history(start=month6, end=today)['Close'].to_numpy()[0]
        three_month_return = (stock_price - values.history(start=month3, end=today)['Close'].to_numpy()[0]) / values.history(start=month3, end=today)['Close'].to_numpy()[0]
        one_month_return = (stock_price - values.history(start=month1, end=today)['Close'].to_numpy()[0]) / values.history(start=month1, end=today)['Close'].to_numpy()[0]

        return [
            ticker,
            stock_price,
            'N/A',
            one_year_return,
            'N/A',
            six_month_return,
            'N/A',
            three_month_return,
            'N/A',
            one_month_return,
            'N/A'
        ]
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        return None

# Use ThreadPoolExecutor to process in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_ticker, ticker, values) for ticker, values in AllYF.items()]
    results = [future.result() for future in as_completed(futures)]

# Filter out any None results due to errors
results = [result for result in results if result is not None]

# Sort the results by ticker
results.sort(key=lambda x: x[0])

# Create the DataFrame
hqm_df = pd.DataFrame(data=results, columns=hqm_columns)
hqm_df


,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,137.32,N/A,0.127700,N/A,0.037404,N/A,-0.014436,N/A,0.079865,N/A
1,AAL,10.62,N/A,-0.365970,N/A,-0.289157,N/A,-0.240343,N/A,-0.038043,N/A
2,AAPL,217.96,N/A,0.109494,N/A,0.139801,N/A,0.257958,N/A,0.005582,N/A
3,ABBV,185.16,N/A,0.237866,N/A,0.150668,N/A,0.156890,N/A,0.096795,N/A
4,ABNB,140.10,N/A,-0.079440,N/A,-0.082515,N/A,-0.136518,N/A,-0.076040,N/A
...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,140.84,N/A,0.249135,N/A,0.242414,N/A,0.062222,N/A,0.070782,N/A
499,YUM,128.05,N/A,-0.069877,N/A,-0.019150,N/A,-0.105734,N/A,-0.012722,N/A
500,ZBH,111.29,N/A,-0.194426,N/A,-0.094614,N/A,-0.085613,N/A,0.050302,N/A
501,ZBRA,325.98,N/A,0.058514,N/A,0.273708,N/A,0.086564,N/A,0.064564,N/A


In [ ]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
